<a href="https://colab.research.google.com/github/singhnidh/Assessment-of-Fruit-Quality-using-Hyperspectral-Imaging-with-the-help-of-Deep-learning/blob/main/ResNet_50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm

#loading images from path
DATADIR = 'D:/archivefruit/fruits-360_dataset/fruits-360/Training'
CATEGORIES = ["Apple Braeburn","Avocado","Banana","Blueberry","Guava","Kiwi","Orange"]
for category in CATEGORIES:
    path = os.path.join(DATADIR, category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img))
        #,cv2.IMREAD_GRAYSCALE
        plt.imshow(img_array)
        plt.show()
        break
    break
#print (room_types)  #what kinds of rooms are in this dataset

print("Types of fruits found: ", len(CATEGORIES))

In [ ]:
IMG_SIZE = 100
batch_size = 100
img_height = 100
img_width = 100

In [ ]:
#building our training data
training_data = []

def create_training_data():
    for category in CATEGORIES:  # do fruits

        path = os.path.join(DATADIR,category)  # create path to fruits
        class_num = CATEGORIES.index(category)  # get the classification  (0 ,1,2,3,4,5,6) different number donate different category of fruit

        for img in tqdm(os.listdir(path)):  # iterate over each image per fruits
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass

create_training_data()

image_count = len(training_data)

In [ ]:
# Build a dataframe
fruits_df = pd.DataFrame(data=training_data, columns=['fruits type', 'image'])
#print(rooms_df.head())
#print(rooms_df.tail())

#Let's check how many samples for each category are present
#print("Total number of fruits in the dataset: ", len(fruits_df))

#fruits_count = fruits_df['fruits type'].value_counts()

#print("fruits in each category: ")
#print(fruit_count)

In [ ]:
#Assigning labels and features
X =[]
y =[]
for features, label in training_data:
    X.append(features)
    y.append(label)
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

#creating own model
#Normalising X and converting labels to categorical data
#X = X.astype('float32') X /= 255 Y = np_utils.to_categorical(y,7) print(Y[100]) print(shape(Y))

In [ ]:
# Transform the image array to a numpy type
X = np.array(X)
X.shape

In [ ]:
#creating own model
#Normalising X and converting labels to categorical data
from keras.utils import np_utils
X = X.astype('float32')
X /= 255
Y = np_utils.to_categorical(y,7)
print(Y[100])
print(Y.shape)

In [ ]:
X = np.array(X)
y=np.array(y)

In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
X, Y = shuffle(X, Y, random_state=1)
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.3, random_state=415)

#inpect the shape of the training and testing.
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

In [ ]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow

from keras.initializers import glorot_uniform
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D


In [ ]:
# x is input, y=F(x)
# identity block simply means input should be equal to output.
#  y = x + F(x)   the layers in a traditional network are learning the true output H(x)
# F(x) = y - x   the layers in a residual network are learning the residual F(x)
# Hence, the name: Residual Block.
def identity_block(A, f, filters, stage, block):
    """
    Arguments:
    A -- input of shape (m, height, width, channel)
    f -- shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network

    Returns:
    A -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Saving the input value.we need this later to add to the output.
    A_shortcut = A
     # First component of main path
    A = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a')(A)
    A = BatchNormalization(axis = 3, name = bn_name_base + '2a')(A)
    A = Activation('relu')(A)

    # Second component of main path (≈3 lines)
    A = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b')(A)
    A = BatchNormalization(axis = 3, name = bn_name_base + '2b')(A)
    A = Activation('relu')(A)

    # Third component of main path (≈2 lines)
    A = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c')(A)
    A = BatchNormalization(axis = 3, name = bn_name_base + '2c')(A)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    A = Add()([A, A_shortcut])
    A = Activation('relu')(A)

    return A

In [ ]:
def convolutional_block(A, f, filters, stage, block, s = 2):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    # Retrieve Filters
    F1, F2, F3 = filters
    # Save the input value
    A_shortcut = A
    # First layer
    A = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a')(A) # 1,1 is filter size
    A = BatchNormalization(axis = 3, name = bn_name_base + '2a')(A)  # normalization on channels
    A = Activation('relu')(A)
    # Second layer  (f,f)=3*3 filter by default
    A = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b')(A)
    A = BatchNormalization(axis = 3, name = bn_name_base + '2b')(A)
    A = Activation('relu')(A)
    # Third layer
    A = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c')(A)
    A = BatchNormalization(axis = 3, name = bn_name_base + '2c')(A)
     ##### SHORTCUT PATH ####
    A_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1')(A_shortcut)
    A_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(A_shortcut)

    # Final step: Add shortcut value here, and pass it through a RELU activation
    A = Add()([A, A_shortcut])
    A = Activation('relu')(A)

    return A

In [ ]:
#creating Resnet 50
#Each ResNet block is either 2 layer deep
def ResNet50(input_shape=(100,100,1), classes=7):
    """
    Implementation of the ResNet50 architecture:
    CONV2D -> BATCHNORM -> RELU -> MAAPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    """
    # Define the input as a tensor with shape input_shape
    A_input = Input(input_shape)
    # Zero-Padding
    A = ZeroPadding2D((3, 3))(A_input) #3,3 padding
    # Stage 1
    A = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(A) #64 filters of 7*7
    A = BatchNormalization(axis=3, name='bn_conv1')(A) #batchnorm applied on channels
    A = Activation('relu')(A)
    A = MaxPooling2D((3, 3), strides=(2, 2))(A) #window size is 3*3
    # Stage 2
    A = convolutional_block(A, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    # convolutional_block is a function defined above. Convolutional_block have 3 layers.
    #filters=[64, 64, 256] first 64 is for 1st layer and 2nd 64 is for 2nd layer and 256 is for 3rd layer of convultional block
    # below are the conv layers from convolutional_block function
    #A = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a')(A)
    #A = Conv2D(F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b')(A)
    #A = Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '2a')(A)
    A = identity_block(A, 3, [64, 64, 256], stage=2, block='b')
    #A = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a')(A)
    #A = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b')(A)
    #A = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c')(A)
    A = identity_block(A, 3, [64, 64, 256], stage=2, block='c')
    #A = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a')(A)
    #A = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b')(A)
    #A = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c')(A)
    ### START CODE HERE ###
    # Stage 3
    A = convolutional_block(A, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    A = identity_block(A, 3, [128, 128, 512], stage=3, block='b')
    A = identity_block(A, 3, [128, 128, 512], stage=3, block='c')
    A = identity_block(A, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    A = convolutional_block(A, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    A = identity_block(A, 3, [256, 256, 1024], stage=4, block='b')
    A = identity_block(A, 3, [256, 256, 1024], stage=4, block='c')
    A = identity_block(A, 3, [256, 256, 1024], stage=4, block='d')
    A = identity_block(A, 3, [256, 256, 1024], stage=4, block='e')
    A = identity_block(A, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    A = convolutional_block(A, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    A = identity_block(A, 3, [512, 512, 2048], stage=5, block='b')
    A = identity_block(A, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL
    A = AveragePooling2D((2,2), name="avg_pool")(A)
     ### END CODE HERE ###
    # output layer
    A = Flatten()(A)
    A = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(A)
    # Create model
    model = Model(inputs = A_input, outputs = A, name='ResNet50')

    return model

In [ ]:
model = ResNet50(input_shape = (100,100,1), classes = 7)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:
import tensorflow as tf
model.fit(train_x, train_y, batch_size = 32, verbose=1,epochs =5,validation_split = 0.3, callbacks=[tf.keras.callbacks.CSVLogger('hisr.csv')])


In [ ]:
#Evaluate the Model
history = pd.read_csv('hisr.csv')
history.head()
plt.figure(figsize=(19,6))
plt.subplot(131)
plt.plot(history.epoch, history.loss, label="loss")
plt.plot(history.epoch, history.val_accuracy, label="val_accuracy")
plt.xlabel('loss')
plt.ylabel('val_accuracy')
plt.legend()

In [ ]:
preds = model.evaluate(test_x, test_y)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]*100),'%')

In [ ]:
#observing its classification report and confusion matrix
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
#predict
y_pred=model.predict(test_x)
y_pred=np.argmax(y_pred,axis=1)

import keras.utils as image from matplotlib.pyplot import imread img_path = 'banana.jpeg' img = image.load_img(img_path, target_size=(100, 100, 1)) x = image.img_to_array(img) x = np.expand_dims(x, axis=0) x = preprocess_input(x) print('Input image shape:', x.shape) my_image = imread(img_path) imshow(my_image) print(model.predict(x))

In [ ]:
test_y_arg=np.argmax(test_y,axis=1)
Y_pred = np.argmax(model.predict(test_x),axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_y_arg, Y_pred))

In [ ]:
from sklearn.metrics import accuracy_score,classification_report
print(classification_report(test_y_arg, Y_pred))